In [2]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
from langchain.llms import OpenAI
import os

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [3]:
MODEL = "gpt-3.5-turbo"
#MODEL = "gpt-4-1106-preview"

OUTPUT_FOLDER = "../generated_GUIs/"
IMAGE_INPUT_FOLDER = "../data/"
S2W = "../data/Screen2Words.csv"
S2W_SUMMARIZED = "../data/s2w_summarized.csv"
S2W_SAMPLE = "../data/s2w_sample.csv"
METADATA = "../data/metadata.csv"

In [4]:
s2w = pd.read_csv(S2W)
s2w.drop(columns=["Unnamed: 0"], inplace=True)

s2w_summarized = pd.read_csv(S2W_SUMMARIZED)
s2w_summarized.drop(columns=["Unnamed: 0"], inplace=True)

s2w_sample = pd.read_csv(S2W_SAMPLE)
s2w_sample.drop(columns=["Unnamed: 0"], inplace=True)

rico = pd.read_csv(METADATA)
rico.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
from prompts import SUMMARY2REQUIRMENTS_ZS, IMPLEMETATION2STRUCTURE_ZS, STRUCTURE2CODE_ZS, FEATURES2IMPLEMENTATION_ZS
from langchain.chains import SequentialChain

def s2r2e2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIRMENTS_ZS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=FEATURES2IMPLEMENTATION_ZS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_ZS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))


In [6]:
from prompts import SUMMARY2STRUCTURE_ZS, STRUCTURE2CODE_ZS
from langchain.chains import SequentialChain

def s2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2STRUCTURE_ZS, output_key="structure",)
    chain_2 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code",)

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [7]:
from prompts import SUMMARY2CODE_ZS

def s2c(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [8]:
def pipeline(summary):
    '''
    response = s2c(summary)
    try: 
        with open("s2c.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("s2c Failed")
        print(e)
    
    response = s2s2c(summary)
    try: 
        with open("s2s2c.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("s2s2c Failed")
        print(e)
    
    response = s2r2s2c_sketch(summary)
    try: 
        with open("s2r2s2c_sketch.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("s2r2s2c Failed")
        print(e)
    '''
    response = s2r2e2s2c(summary)
    try: 
        with open("s2r2s2c.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("s2r2s2c Failed")
        print(e)

In [6]:
pipeline("The mobile screen is part of a language learning application that asks the user to translate a given text sentence.")

In [11]:
rico[rico["Play Store Name"]=="Duolingo: Learn Languages Free"]

,App Package Name,Play Store Name,Category,Average Rating,Number of Ratings,Number of Downloads,Date Updated,UI Number,Interaction Trace Number,UI Number in Trace,split
15465,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27727,1,2846,train
15466,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27729,1,1834,train
15467,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27730,1,3373,train
15468,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27731,1,2922,train
15469,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27732,1,2155,train
15470,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27733,1,3800,train
15471,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27734,1,1814,train
15472,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27735,1,2377,train
15473,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27737,1,732,train
15474,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27738,1,4017,train


In [10]:
rico[rico["UI Number"]==27729]

,App Package Name,Play Store Name,Category,Average Rating,Number of Ratings,Number of Downloads,Date Updated,UI Number,Interaction Trace Number,UI Number in Trace,split
15466,com.duolingo,Duolingo: Learn Languages Free,Education,4.7,4449253,"50,000,000 - 100,000,000","May 11, 2017",27729,1,1834,train
